In [1]:
%matplotlib inline

from __future__ import division
import sys
import csv
csv.field_size_limit(sys.maxsize)

import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
import shapely.wkt
import shapely.affinity
import cv2

IMG_ID = '6070_2_3'
LABELS = ['7', '8']

In [3]:
class Image:
    
    def __init__(self, img_id, labels):
        self.img_id = img_id
        self.labels = labels
        
        self.read_images()
        self.read_train_polygons()
        self.read_train_rasters()
    
    def read_images(self):
        self.img_rgb = tiff.imread('water_body_extraction/input/%s.tif'%self.img_id).transpose((1, 2, 0))
        self.img_a = tiff.imread('water_body_extraction/input/%s_A.tif'%self.img_id).transpose((1, 2, 0))
        self.img_m = tiff.imread('water_body_extraction/input/%s_M.tif'%self.img_id).transpose((1, 2, 0))
        self.img_p = tiff.imread('water_body_extraction/input/%s_P.tif'%self.img_id)
        (self.h, self.w, _) = self.img_rgb.shape
        
        self.img_a_full = self._upscale_image_to_full_size(self.img_a)
        self.img_m_full = self._upscale_image_to_full_size(self.img_m)    
    
    def _upscale_image_to_full_size(self, img):
        (h, w, _) = img.shape
        img_full = scipy.ndimage.interpolation.zoom(img, (self.h / h, self.w / w, 1), order = 1)
        return img_full
    
    def read_train_polygons(self):
        self.train_polygons = {}
        
        for _im_id, _x, _y in csv.reader(open('data/grid_sizes.csv')):
            if _im_id == IMG_ID:
                x_max, y_min = float(_x), float(_y)
                break      
        x_scaler = self.w * (self.w / (self.w+1)) / x_max
        y_scaler = self.h * (self.h / (self.h+1)) / y_min   
        
        for _im_id, _poly_type, _poly in csv.reader(open('data/train_wkt_v4.csv')):                
            if _im_id == self.img_id and _poly_type in self.labels:
                p = shapely.affinity.scale(shapely.wkt.loads(_poly), xfact = x_scaler, yfact = y_scaler, origin = (0,0,0)) 
                self.train_polygons[_poly_type] = p

    def read_train_rasters(self):
        self.img_masks = {}
        for k in self.train_polygons:
            train_polygon = self.train_polygons[k]
            img_mask = np.zeros((self.h, self.w), np.uint8)
            int_coords = lambda x: np.array(x).round().astype(np.int32)
            exteriors = [int_coords(poly.exterior.coords) for poly in train_polygon]
            interiors = [int_coords(pi.coords) for poly in train_polygon
                         for pi in poly.interiors]
            cv2.fillPoly(img_mask, exteriors, 1)
            cv2.fillPoly(img_mask, interiors, 0)
            self.img_masks[k] = img_mask        
    
i = Image(IMG_ID, LABELS)
print(i.img_masks)

    

{'8': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), '7': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}


In [10]:
def upscale_img(img, to_h, to_w):
    (orig_h, orig_w, _) = img.shape
    return scipy.ndimage.interpolation.zoom(img, (to_h / orig_h, to_w / orig_w, 1), order = 3)

In [23]:
masks = [(gen_mask(polys, h, w)) for polys in gen_polygons(h, w)]
full_mask = np.bitwise_or(masks[0], masks[1])
full_mask_in_3d = full_mask.reshape(h, w, 1).repeat(3, 2)
#masked = np.multiply(img_rgb, full_mask_in_3d)
#plt.imshow(masked)